#Load Dataset

In [1]:
!wget -O LLM-ReDial-2024.zip "https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1"
!unzip LLM-ReDial-2024.zip
!unzip LLM_Redial.zip
!rm LLM_Redial.zip
!rm LLM-ReDial-2024.zip
!rm -rf __MACOSX

--2024-12-02 21:44:14--  https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucdae89d9e32f79d94adf6dfcfa2.dl.dropboxusercontent.com/cd/0/inline/Cfg941DRJivfVHDCf0GYoRSrJi8eTOYPoVSvmrNqkZbqgEkwRIDdCvbFi7vTXqC1lJwoIaRMUjLatw8HRToyq95_9jM1kxIYzxwXb-IUGrbVSrGw5wMvKy7waJKSU7p4rzU/file?dl=1# [following]
--2024-12-02 21:44:14--  https://ucdae89d9e32f79d94adf6dfcfa2.dl.dropboxusercontent.com/cd/0/inline/Cfg941DRJivfVHDCf0GYoRSrJi8eTOYPoVSvmrNqkZbqgEkwRIDdCvbFi7vTXqC1lJwoIaRMUjLatw8HRToyq95_9jM1kxIYzxwXb-IUGrbVSrGw5wMvKy7waJKSU7p4rzU/file?dl=1
Resolving ucdae89d9e32f79d94adf6dfcfa2.dl.dropboxusercontent.com (ucdae89d9e32f79d94adf6dfcfa2.dl.dropboxusercontent.com)... 162.125.81.15, 2620:100:6030:15::

#Load HF model

In [2]:
from huggingface_hub import notebook_login

notebook_login()



In [6]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-b4d7dd08-c583-dc1e-9c0f-c7c2d80e1195)


In [63]:
!pip install transformers[torch]
!pip install bitsandbytes
!pip install peft
!pip install sentence-transformers

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [5]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [8]:
def generate_zeroshot(conversation_id, conversations):
    conversation = "\n".join(conversations[conversation_id]["messages"])
    content = (
        "Pretend you are a movie recommender system. "
        "I will give you a conversation between a human and assistant. "
        "Based on the conversation, you reply me with 10 recommendations "
        f"without extra sentences. Here is the conversation: {conversation}."
    )
    chat = [
        {"role": "user", "content": content}
    ]
    return tokenizer.apply_chat_template(chat, tokenize=False)

In [9]:
from Tools import read_user_data
path_movie_data = "/content/Movie/final_data.jsonl"

In [10]:
import json
item_map_path = "/content/Movie/item_map.json"
item_map = json.load(open(item_map_path,'r'))
user_map_path = "/content/Movie/user_ids.json"
user_map = json.load(open(user_map_path,'r'))

In [11]:
conversations_path = "/content/Movie/Conversation.txt"

conversations = []
current_conversation = []
conversation_id = 0
with open(conversations_path, 'r') as file:
    for line in file:
        line = line.strip()
        if not line:
            continue

        if line.isdigit():
            if current_conversation:
                conversations.append({
                    'conversation_id': conversation_id,
                    'messages': current_conversation
                })
            conversation_id = int(line)
            current_conversation = []
        else:
            current_conversation.append(line)

    if current_conversation:
        conversations.append({
            'conversation_id': conversation_id,
            'messages': current_conversation
        })

In [36]:
import random
user_ids = list(user_map.keys())
sample_percent = 0.01 # 10%

sample_size = int(sample_percent * len(user_ids))
sampled_users = random.sample(user_ids,sample_size)

In [50]:
dataset = []
for user_id in user_ids:
  user_data = read_user_data(path_movie_data, user_id)
  for conv_number, conversation in enumerate(user_data['Conversation']):
    conversation_id = conversation[f"conversation_{conv_number+1}"]["conversation_id"]
    prompt = generate_zeroshot(conversation_id, conversations)
    dataset.append({
        "user_id": user_id,
        "conversation_id": conversation_id,
        "user_might_like": user_data['user_might_like'],
        "prompt": prompt
    })

In [54]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)
print("Train prompts:", len(train_set))
print("Test prompts:", len(test_set))

Train prompts: 8071
Test prompts: 2018


In [55]:
prompts = [item['prompt'] for item in train_set[:10]]
results = text_generator(prompts, max_new_tokens=200, return_full_text=False)


In [56]:
answers = [result[0]['generated_text'] for result in results]
for answer in answers:
  print(answer)


 Here are 10 movie recommendations based on the user's preferences:

1. "The Big Chill" - A drama about a group of college friends who reunite after a tragedy.
2. "California Suite" - A comedy-drama about different couples staying in the same hotel suite in Beverly Hills.
3. "The Holiday" - A romantic comedy about two women who swap homes for a week to escape from their problems.
4. "The Way We Were" - A romantic drama about a liberal activist and a conservative man who fall in love.
5. "The American President" - A romantic political drama about a widowed president and a lobbyist.
6. "The Royal Tenenbaums" - A comedy-drama about a dysfunctional family of child prodigies.
7. "The Graduate" - A comedy-drama about a college graduate who becomes entangled in a romantic
 Based on the conversation, here are 10 movie recommendations similar to "Below" and "Ronin":

1. The Abyss (1989) - A science fiction film that combines elements of horror, mystery, and suspense.
2. The Thing (1982) - A hor

In [57]:
import re

def parse_recommendations(answer):
  pattern = r'\d+\.\s+"*([^"]+?)"*(?:\s\(\d{4}\))?(?:\s*-\s*.+)?(?:\s*\(\d-\d\))?$'
  recommendations = re.findall(pattern, answer, re.MULTILINE)
  return recommendations

In [58]:
import numpy as np

def recall_at_k(r, k, m):
    return (np.asarray(r)[:k] != 0).sum() / m

def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.

def idcg_at_k(k):
    return dcg_at_k(np.ones(k), k)

def ndcg_at_k(r, k, max_relevant):
    idcg = idcg_at_k(min(k, max_relevant))
    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

In [64]:
from sentence_transformers import SentenceTransformer

def embedding_init():
  sbert = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
  return sbert

def similarity_score(sbert, movies):
  embed1 = sbert.encode(movies[0])
  embed2 = sbert.encode(movies[1])
  return sbert.similarity(embed1,embed2).numpy()[0][0]

In [72]:
def evaluate_model(dataset, answers, sbert):
    assert len(prompts) == len(answers)

    mean_ndcg = 0
    mean_recall = 0
    recommendations_count = 0
    for i in range(len(answers)):
       recommendations = parse_recommendations(answers[i])
       if len(recommendations) != 10:
         continue
       user_might_like = [item_map[id] for id in dataset[i]["user_might_like"]]
       similarities = [similarity_score(sbert, [user_might_like[j], recommendations[j]]) for j in range(len(answers))]
       rel_vector = np.array([1 if score >= 0.75 else 0 for score in similarities])

       mean_ndcg += ndcg_at_k(rel_vector, 20, len(user_might_like))
       mean_recall += recall_at_k(rel_vector, 20, len(user_might_like))
       recommendations_count += 1

    mean_ndcg /= recommendations_count
    mean_recall /= recommendations_count

    return mean_ndcg, mean_recall

In [73]:
sbert = embedding_init()
ndcg, recall = evaluate_model(test_set[:10], answers, sbert)
print(f"NDCG: {ndcg:.4f}")
print(f"Recall: {recall:.4f}")

NDCG: 0.0000
Recall: 0.0000
